<font size=5>**案例**：东方财富研报基本信息爬虫</font>  

In [3]:
import requests
from bs4 import BeautifulSoup
import re
from lxml import etree
import pandas as pd
import numpy as np
import time
import random

&emsp;&emsp;首先使用chorme打开东方财富网个股研报网站：http://data.eastmoney.com/report/stock.jshtml。
<div align="center">

<img src="pic\东财研报page1.png" alt="图片无法显示时的提示"  width="49%" height="50%" >
<img src="pic\东财研报page2.png" alt="图片无法显示时的提示"  width="49%" height="50%">
 </div>  
&emsp;&emsp;可以看到，网址http://data.eastmoney.com/report/stock.jshtml中并没有参数指明当前请求第一页的数据，我们单击下一页，上方网址不变，因此可以确定，东财采用异步加载的方式加载数据。也就是说上方的网址并不是请求研报数据的最终网址，我们需要抓包来找到访问数据的url以加载研报数据。
<div align="center">

<img src="pic\fetch0.png" alt="图片无法显示时的提示"  width="30%" height="30%" >
<img src="pic\fetch1.png" alt="图片无法显示时的提示"  width="59%" height="59%">
 </div>
 &emsp;&emsp;很显然这个list开头的，大小为13.4kb的文件，符合我们的需求，我们单击后可以看到这13.4kb中确实包含了我们需要的研报基本信息。下面我们来解析请求的方式与请求的网址。
 <div align="center">
<img src="pic\fetch标头.png" alt="图片无法显示时的提示"  width="50%" height="50%" >
 </div>
&emsp;&emsp;那么很明显，请求的方式是get，真正调取数据的网址是http://reportapi.eastmoney.com/report/list 问号后面的是提交的参数，由于get的特性，参数会在url中显示出来。为了探究参数的含义，我们尝试抓取不同页数的url进一步分析。</br>  
&emsp;&emsp;第三页： http://reportapi.eastmoney.com/report/list?cb=datatable2678479&industryCode=*&pageSize=50&industry=*&rating=&ratingChange=&beginTime=2019-11-12&endTime=2021-11-12&pageNo=3&fields=&qType=0&orgCode=&code=*&rcode=&p=3&pageNum=3&pageNumber=3&_=1636691974800  </br>  
&emsp;&emsp;第四页： http://reportapi.eastmoney.com/report/list?cb=datatable66050&industryCode=*&pageSize=50&industry=*&rating=&ratingChange=&beginTime=2019-11-12&endTime=2021-11-12&pageNo=4&fields=&qType=0&orgCode=&code=*&rcode=&p=4&pageNum=4&pageNumber=4&_=1636691974802  </br>  
&emsp;&emsp;解析参数，industryCode=*&pageSize=50&industry=*&rating=&ratingChange=&beginTime=2019-11-12&endTime=2021-11-12&fields=&qType=0&orgCode=&code=*&rcode= 这些参数显然是不变，或为空的，照抄即可 。</br> 
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;pageNo=4&p=4&pageNum=4&pageNumber=4这四个参数其实都是在描述当前页码为第四页一回事一起改即可。</br> 
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;_=1636691974802这个下划线参数表示时间戳，时间戳无非就是约定以某一日为起点，然后年，日，时，分，秒统一转化为比起点多了多少分钟或多少秒，是计算机认识时间的基本方式，也是各种时间对象的底层逻辑，获取当前的时间戳可以通过掉包实现。根据参数时间戳的位数，可以知道这是毫秒级时间戳。</br> 
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;最后提一下cb参数，可以看到cb参数的值为datatable加上一串数字。我尝试随机修改个别数字访问网页，发现完全不影响对数据的请求，猜测这个参数可能和端口的概念有点类似，通过不用的id的数据表访问，也许能降低服务器的负担（个人猜想）。</br> 
&emsp;&emsp;分析完所有的参数，就可以构造完备的url。下面我们开始发送get请求访问页面。


In [4]:
# 这段代码错了,为什么？
urls = [ 'http://reportapi.eastmoney.com/report/list?cb=datatable%d&industryCode=*&pageSize=50&industry=*&\
    rating=&ratingChange=&beginTime=2019-11-12&endTime=2021-11-12&pageNo=%d&fields=&qType=0&orgCode=&code=*&rcode=&p=%d&\
        pageNum=%d&pageNumber=%d&_=%d'%(random.randint(1000000,2678479),i,i,i,i,int(round(time.time() * 1000))) for i in range(1,6) ]

urls

['http://reportapi.eastmoney.com/report/list?cb=datatable1575462&industryCode=*&pageSize=50&industry=*&    rating=&ratingChange=&beginTime=2019-11-12&endTime=2021-11-12&pageNo=1&fields=&qType=0&orgCode=&code=*&rcode=&p=1&        pageNum=1&pageNumber=1&_=1636977431008',
 'http://reportapi.eastmoney.com/report/list?cb=datatable1081273&industryCode=*&pageSize=50&industry=*&    rating=&ratingChange=&beginTime=2019-11-12&endTime=2021-11-12&pageNo=2&fields=&qType=0&orgCode=&code=*&rcode=&p=2&        pageNum=2&pageNumber=2&_=1636977431008',
 'http://reportapi.eastmoney.com/report/list?cb=datatable1146518&industryCode=*&pageSize=50&industry=*&    rating=&ratingChange=&beginTime=2019-11-12&endTime=2021-11-12&pageNo=3&fields=&qType=0&orgCode=&code=*&rcode=&p=3&        pageNum=3&pageNumber=3&_=1636977431008',
 'http://reportapi.eastmoney.com/report/list?cb=datatable1469338&industryCode=*&pageSize=50&industry=*&    rating=&ratingChange=&beginTime=2019-11-12&endTime=2021-11-12&pageNo=4&fields=&qTyp

In [5]:
# 正确的代码
urls = ( 'http://reportapi.eastmoney.com/report/list?cb=datatable%d&industryCode=*&pageSize=50&industry=*&\
    rating=&ratingChange=&beginTime=2019-11-12&endTime=2021-11-12&pageNo=%d&fields=&qType=0&orgCode=&code=*&rcode=&p=%d&\
        pageNum=%d&pageNumber=%d&_=%d'%(random.randint(1000000,2678479),i,i,i,i,int(round(time.time() * 1000))) for i in range(1,6) )

In [7]:
#尝试发送访问请求并返回状态码
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}
html = requests.get(url=next(urls),headers=headers)
html.status_code

200

In [ ]:
#简单地展示一下返回的内容
html.text[:200] +html.text[-200:] 

'datatable2340185({"hits":57365,"size":50,"data":[{"title":"三季度业绩超预期，传统淡季交出好的答卷","stockName":"凯立新材","stockCode":"688269","orgCode":"80000124","orgName":"天风证券股份有限公司","orgSName":"天风证券","publishDate":"202Rkij+X3BdpvZ6GgoDZl8MEssCNxS+c78=","sRatingName":"买入","sRatingCode":"0201","market":"SHANGHAI","authorID":["11000278640"],"count":7,"orgType":"white"}],"TotalPage":1148,"pageNo":1,"currentYear":2021})'

&emsp;&emsp;查看text的返回值，可以看到并没有那些超文本标记语言的语法，反而像是返回了一个字符串形式的字典，如果把开头的datatable几个字符去掉的话。因此可以尝试使用json解析text，如果对json没有概念，也可以使用eval凑合，如果连eval也不会不百度，建议从入门到放弃。</br>
&emsp;&emsp;使用json的格式化校验工具可以让我们对这个巨大的字典的结构有一个清晰的认知：
 <div align="center">
<img src="pic\json格式化校验.png" title="json格式化校验"  width="50%" height="50%" >
 </div>


In [8]:
#使用eval凑合
html_dict = eval(html.text[17:-1])
data_list = html_dict['data'] #data_list是个列表，列表里每一个元素都是一个大字典。

df = pd.DataFrame(None,columns=['title','stockName','stockCode','orgSName','predictNextTwoYearEps'])
for i in range(len(data_list)):
    #提取标题，股票名，股票代码，证券公司，两年后eps 这五项信息
    title,stockName,stockCode,orgSName,predictNextTwoYearEps = data_list[i]['title'],data_list[i]['stockName'],\
        data_list[i]['stockCode'],data_list[i]['orgSName'],data_list[i]['predictNextTwoYearEps']

    df = df.append(pd.DataFrame(np.array([[title,stockName,stockCode,orgSName,predictNextTwoYearEps]]),columns=['title','stockName','stockCode','orgSName','predictNextTwoYearEps']))
df

,title,stockName,stockCode,orgSName,predictNextTwoYearEps
0,三季度业绩超预期，传统淡季交出好的答卷,凯立新材,688269,天风证券,3.9400000000
0,募投缓解产能瓶颈，多品种布局利于长期发展,恒顺醋业,600305,中银证券,0.3180000000
0,PCB行业持续景气公司收入快速增长，PET镀铜设备处于需求爆发前夜,东威科技,688700,中银证券,2.5730000000
0,工商业稳健发展，新业务有望逐步贡献业绩,华东医药,000963,南京证券,
0,转型锂业赛道 价值获得重估,盛新锂能,002240,申港证券,1.5100000000
0,72页深度报告：创新驱动，研运共进,吉比特,603444,国海证券,28.6000000000
0,第一名中标移动机房空调集采，产品力持续提升,英维克,002837,东吴证券,1.2400000000
0,恩捷股份：携手宁德，强强联合，龙头地位凸显,恩捷股份,002812,东吴证券,6.8100000000
0,公司简评报告：新客户逐渐放量，Q3收入同环比高增,光弘科技,300735,首创证券,0.7900000000
0,Q3业绩创历史新高，MIM龙头持续受益行业发展,精研科技,300709,国盛证券,2.1300000000


&emsp;&emsp;由于我们的任务目标是爬取5页，也就是我们的目标url有5个，我们需要在外面再嵌套一个循环批量处理，所以完整的爬虫代码如下：

In [ ]:
urls = ( 'http://reportapi.eastmoney.com/report/list?cb=datatable%d&industryCode=*&pageSize=50&industry=*&\
    rating=&ratingChange=&beginTime=2019-11-12&endTime=2021-11-12&pageNo=%d&fields=&qType=0&orgCode=&code=*&rcode=&p=%d&\
        pageNum=%d&pageNumber=%d&_=%d'%(random.randint(1000000,2678479),i,i,i,i,int(round(time.time() * 1000))) for i in range(1,6) )

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

df = pd.DataFrame(None,columns=['title','stockName','stockCode','orgSName','predictNextTwoYearEps']) #保存最终内容的df要在循环外创建

for url in urls:
    html = requests.get(url=url,headers=headers)
    if html.status_code==200: #状态码200表示访问成功
        html_dict = eval(html.text[17:-1])
        data_list = html_dict['data'] #data_list是个列表，列表里每一个元素都是一个大字典。

        
        for i in range(len(data_list)):
            #提取标题，股票名，股票代码，证券公司，两年后eps 这五项信息
            title,stockName,stockCode,orgSName,predictNextTwoYearEps = data_list[i]['title'],data_list[i]['stockName'],\
                data_list[i]['stockCode'],data_list[i]['orgSName'],data_list[i]['predictNextTwoYearEps']

            df = df.append(pd.DataFrame(np.array([[title,stockName,stockCode,orgSName,predictNextTwoYearEps]]),\
                columns=['title','stockName','stockCode','orgSName','predictNextTwoYearEps']))

df.reset_index(drop=True).to_excel('research_report.xlsx')
df.info()
        

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 0
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   title                  250 non-null    object
 1   stockName              250 non-null    object
 2   stockCode              250 non-null    object
 3   orgSName               250 non-null    object
 4   predictNextTwoYearEps  250 non-null    object
dtypes: object(5)
memory usage: 11.7+ KB


In [9]:
html.cookies

<RequestsCookieJar[]>

In [10]:
for cookie in html.cookies:
    print(cookie)